In [1]:
#pip install import-ipynb

In [306]:
import string
import numpy as np
import import_ipynb
import pixigame_modified_play_wordle_with_agent_loop
import random
from wordle import Wordle
import re
import time
from collections import namedtuple
from collections import defaultdict
import pandas as pd

In [527]:
import alphabet

BUNNY


In [ ]:
alphabet = string.ascii_uppercase

In [ ]:
alphabet

In [8]:
def load_word_set(path: str):
    word_set = set()
    with open(path, "r") as f:
        for line in f.readlines():
            word = line.strip().upper()
            word_set.add(word)
    return word_set

# feedback function

In [163]:
# word_set = load_word_set("data/wordle_words.txt") # this is a 10k 5-letter word list 
#                                                     this is not representative of actual wordle game play
#
word_set = load_word_set("data/wordle_la.txt") # this is the reduced possible wordle word list

np_word_list = np.array(list(word_set))  # this makes the word_set into an np.array for my processing
secret = random.choice(list(word_set))
wordle = Wordle(secret)  # this launches the wordle game and provides the secret to the game
print(secret)  # this shows us the secret word

ATTIC


In [ ]:
np_word_list[0]

In [ ]:
len(np_word_list)

In [136]:
my_secret = wordle.secret  # this is the actual secret word from the wordle game instance that is running
my_guess = "STINT"  # this is my guess word for the feedback function

In [238]:
# this function needs to be provided a guess that is 5 letters 
# and a word set that is a numpy array list of 5 letter words. 
# This function can be provided a secret which is the secret word it is comparing the guess to
# The default for this function can get this secret from wordle.secret  secret=wordle.secret
# 
# this function has been written with default settings that can be overridden
# these function are the secret,np_word_list, printing, and my_return_complex
# 
# secret is used in the master agent game play to provide the game: secret=wordle.secret
# 
# np_word_list is the vocabulary list the function is working with: np_word_list=np_word_list 
#      you can provide a different list by specifying a new list: np_word_list=np_other_list
# 
# printing controls whether the debuging printing is done or not: printing =False can be set to true: printing=True
#
# my_return_complex controls the return function of the function: my_return_complex=False
# the default returns a size integer.  This is leveraged by an np.vectorize function that requires a simple output
# 


# the my_return_complex=True setting will return a namedtuple
# in this mode the return is a named tuple with these named tuples: 
# green_result,yellow_result,grey_list,not_green_list,regexstring_out,mylength,myregexresult
#
# 
# this is run like this:
# np_vocab_reduction = np.vectorize(feedback)
# mylengtharray = np_vocab_reduction(np_word_list,secret = my_secret,printing=False)
# mylengtharray[0:50]
#
# array([  1, 524,  59,   5, 172,  53, 207,  36, 284,  78, 357,  26,  12,
#          5, 459, 970,  76,  19, 208, 159, 237,   4, 169,  98, 103, 290,
#         53,  21, 212, 238,  68,  16, 126, 129, 172, 218, 145,  11, 228,
#          2,  84, 302, 116,  15,  13,  19, 146, 285,  24,  19])
#
# np_word_list[mylengtharray == 1]
# array(['SATIN', 'STERN', 'SALON', 'SLAIN', 'STRIP', 'STAIR', 'STEIN',
#        'PATSY', 'STINT', 'BASIC', 'SCANT', 'PATIO', 'STONE', 'FAINT',
#        'TRAIN', 'STANK', 'STAIN', 'SATYR', 'PAINT', 'SLANT', 'STAND',
#        'WAIST', 'STING', 'STONY', 'UNTIE', 'BASIS', 'STUNT', 'BASIL',
#        'BASIN', 'SAINT', 'STINK', 'STAID', 'UNTIL', 'TAINT', 'MASON',
#        'RATIO', 'ARTSY', 'TITAN', 'SNAIL', 'ANTIC', 'CABIN'], dtype='<U5')
#
# running this function in this way is done like this:
# my_results = feedback('bread',secret = my_secret,np_word_list=np_word_list,printing=True,my_return_complex=True)
#
# this provides the capability to run this kind of command:
# print(my_results.myregexresult)
# each of these results can be used: 
#      my_results.green_result                00100 
#      my_results.yellow_result               00001
#      my_results.grey_list                   ['B', 'R', 'E', 'D']
#      my_results.not_green_list              [0, 1, 2, 3, 4]
#      my_results.regexstring_out             (?=[^BRED][^BRED][^BRED][^ABRED][^BRED])(?=.*A.*)
#      my_results.mylength                    235
#      my_results.myregexresult               ['SATIN' 'FLASK' 'TAUNT' 'FLANK' 'SALVO' 'FAITH' 'AMITY' 'CAULK' 'LOAMY'
#                                             ...
#                                             'CANNY']

def feedback(guess ,secret=my_secret, np_word_list=np_word_list,printing=False,my_return_complex=False):
    guess = guess.upper()
    if printing == True: print("Top of the feedback function\n","secret",secret,"guess",guess,wordle.attempts ) 
    #secret = wordle.secret
            
    # this makes green results
    green_result = ""
    maskedword = ""
    myregexstring = [[],[],[],[],[]]

    for i in range(len(secret)):
        same = (guess[i] in secret[i])
        green_result += str(int(same))
        if same:
            maskedword += "*"
            myregexstring[i] = guess[i]
        else:
            maskedword += secret[i]
    if printing == True: print("maskedword",maskedword)
    if printing == True: print("green result",green_result)

    # This makes yellow results
    yellow_result = ""
    myisinstring = ""
    myisinfstring = '(?=.*{}.*)'

    for i in range(len(maskedword)):
        isin = (guess[i] in maskedword)
        if isin:
            myregexstring[i].append(guess[i])
            myisinstring += f'(?=.*{guess[i]}.*)'

        yellow_result += str(int(isin))
    if printing == True: print("yellow_result",yellow_result)

    # greylist is a list of letters that are not green and are not yellow
    # the idea is that the letter has not been filtered out
    # not_green_list is a list of positions that are not green

    grey_list = []
    not_green_list = []
    for i in range(len(yellow_result)):
        #print(green_result[i]== "0", yellow_result[i] == "0")
        if green_result[i] == "0":
            not_green_list.append(i)
        if green_result[i] == "0" and yellow_result[i] == "0" and guess[i] not in grey_list:
            if printing == True: print(guess[i])
            grey_list.append(guess[i])
        if printing == True: print(i,secret[i],guess[i],green_result[i],yellow_result[i],grey_list,not_green_list)
    if printing == True: print("grey_list",grey_list)
    if printing == True: print("not_green_list",not_green_list)


    # this builds a myregexstring that is used to produce a regular expression string
    # that is used to filter out words from the vocabulary list based on the restrictions of the wordle feedback

    for i in not_green_list:
        for grey in grey_list:
            if not grey in myregexstring[i]:
                myregexstring[i].append(grey)
    if printing == True: print("myregexstring",myregexstring)
    if printing == True: print("myisinstring",myisinstring)

    #wordlist_five

    finalregexstring = ""
    for i in myregexstring:
        if type(i) == list:
            list_string = "".join(i)
            f_list_string = "[^{}]".format(list_string)
            if printing == True: print("list_string",list_string,f_list_string)
            finalregexstring += f_list_string
        elif type(i) == str:
            finalregexstring += i
        if printing == True: print(i,type(i),str(i),finalregexstring)
    regexstring_out = "(?={}){}".format(finalregexstring,myisinstring)
    if printing == True: print(regexstring_out)
    
    ### this is the original word set as a np array.
    ### the first use of this is to evaluate the reduction of the entire word set
    ### i'm commenting this out so that the word set can be provided to the function
    ### This provides a way to pass in a reduced data set
    #np_word_list = np.array(list(word_set))
    
    
    myregexresult = np_word_list[(list(map(lambda x: bool(re.match(regexstring_out,x)),np_word_list)))]
    if printing == True: print(myregexresult)
    mylength = len(myregexresult)
    if printing == True: print(mylength)
    #return(green_result,yellow_result,grey_list,not_green_list,regexstring_out,mylength,myregexresult)


    myresults = namedtuple("myresults",["green_result","yellow_result","grey_list","not_green_list","regexstring_out","mylength","myregexresult"])
    results = myresults(green_result,yellow_result,grey_list,not_green_list,regexstring_out,mylength,myregexresult)
    if my_return_complex == True:
        return(results)
    else:
        return(mylength)

    

# This runs the script as default my_return_complex which is my_return_complex=False


In [247]:
# This runs the script as default my_return_complex which is my_return_complex=False
# this results in a vocabulary length as the result
# the secret word is the first word in the vocabulary
# this is intended to provide an opportunity to produce a matrix of vocabulary length when run in a loop
# using the np_vocab_reduction = np.vectorize(feedback)
# mylengtharray = np_vocab_reduction(np_word_list,secret = my_secret,printing=False)
# these will result in a numpy array for each word in the vocabulary as the secret word
# compared to all of the other words as guesses.  the result is the length of the vocabulary resulting
# turn off the printing and this takes around 9 seconds to run on the ~2000 vocabulary words
# 
my_secret = np_word_list[0]
mylength = feedback('bread',secret = my_secret,np_word_list=np_word_list,printing=True)
print(mylength)

Top of the feedback function
 secret SATIN guess BREAD []
maskedword SATIN
green result 00000
yellow_result 00010
B
0 S B 0 0 ['B'] [0]
R
1 A R 0 0 ['B', 'R'] [0, 1]
E
2 T E 0 0 ['B', 'R', 'E'] [0, 1, 2]
3 I A 0 1 ['B', 'R', 'E'] [0, 1, 2, 3]
D
4 N D 0 0 ['B', 'R', 'E', 'D'] [0, 1, 2, 3, 4]
grey_list ['B', 'R', 'E', 'D']
not_green_list [0, 1, 2, 3, 4]
myregexstring [['B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D'], ['A', 'B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D']]
myisinstring (?=.*A.*)
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED][^BRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED][^BRED][^BRED]
list_string ABRED [^ABRED]
['A', 'B', 'R', 'E', 'D'] <class 'list'> ['A', 'B', 'R', 'E', 'D'] [^BRED][^BRED][^BRED][^ABRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', '

# This runs the script as default my_return_complex which is my_return_complex=True

In [248]:
# This runs the script as default my_return_complex which is my_return_complex=True
# this results in the results being returned as a named tuple
# print(my_results.myregexresult)
# ['SATIN' 'FLASK' 'TAUNT' 'FLANK' 'SALVO' 'FAITH' 'AMITY' 'CAULK' 'LOAMY'
# ...
# 'CANNY']
#
print(my_secret)
my_results = feedback('bread',secret = my_secret,np_word_list=np_word_list,printing=True,my_return_complex=True)

SATIN
Top of the feedback function
 secret SATIN guess BREAD []
maskedword SATIN
green result 00000
yellow_result 00010
B
0 S B 0 0 ['B'] [0]
R
1 A R 0 0 ['B', 'R'] [0, 1]
E
2 T E 0 0 ['B', 'R', 'E'] [0, 1, 2]
3 I A 0 1 ['B', 'R', 'E'] [0, 1, 2, 3]
D
4 N D 0 0 ['B', 'R', 'E', 'D'] [0, 1, 2, 3, 4]
grey_list ['B', 'R', 'E', 'D']
not_green_list [0, 1, 2, 3, 4]
myregexstring [['B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D'], ['A', 'B', 'R', 'E', 'D'], ['B', 'R', 'E', 'D']]
myisinstring (?=.*A.*)
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED][^BRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 'E', 'D'] [^BRED][^BRED][^BRED]
list_string ABRED [^ABRED]
['A', 'B', 'R', 'E', 'D'] <class 'list'> ['A', 'B', 'R', 'E', 'D'] [^BRED][^BRED][^BRED][^ABRED]
list_string BRED [^BRED]
['B', 'R', 'E', 'D'] <class 'list'> ['B', 'R', 

In [533]:
my_results.green_result

'00000'

In [534]:
my_results.grey_list

['B', 'R', 'E', 'D']

In [537]:
my_results.mylength

235

In [538]:
my_results.myregexresult

array(['SATIN', 'FLASK', 'TAUNT', 'FLANK', 'SALVO', 'FAITH', 'AMITY',
       'CAULK', 'LOAMY', 'COMMA', 'MAGIC', 'STALL', 'ALIGN', 'PLAZA',
       'SLASH', 'CAPUT', 'ALOOF', 'MAXIM', 'CLACK', 'TANGO', 'CHANT',
       'CLANK', 'LOATH', 'FLASH', 'QUAIL', 'SCALP', 'SALON', 'AWAIT',
       'SLAIN', 'FLACK', 'NASTY', 'PASTA', 'PALSY', 'ALOFT', 'HASTY',
       'FANCY', 'CHAFF', 'TONGA', 'GAWKY', 'AXION', 'SASSY', 'AMONG',
       'PANIC', 'LAUGH', 'MOCHA', 'SMACK', 'GAMMA', 'NANNY', 'PLANT',
       'ALPHA', 'SHACK', 'SWASH', 'GAMUT', 'SPANK', 'GIANT', 'CLAIM',
       'PIZZA', 'GAUNT', 'SALTY', 'ANVIL', 'KAPPA', 'GUAVA', 'PATSY',
       'NINJA', 'QUOTA', 'KOALA', 'ANNUL', 'MACHO', 'PASTY', 'SWATH',
       'PSALM', 'SPAWN', 'SAUNA', 'HAVOC', 'CHAMP', 'KNACK', 'WACKY',
       'SCALY', 'VOILA', 'MAMMY', 'SCANT', 'AWFUL', 'PATIO', 'AWASH',
       'SHANK', 'MATCH', 'JAUNT', 'TAWNY', 'SNACK', 'CANON', 'FAINT',
       'VAULT', 'SALSA', 'WAGON', 'PANSY', 'JUNTA', 'STANK', 'SAPPY',
       'CLASS', 'AUN

In [539]:
my_results.not_green_list

[0, 1, 2, 3, 4]

In [540]:
my_results.regexstring_out

'(?=[^BRED][^BRED][^BRED][^ABRED][^BRED])(?=.*A.*)'

In [541]:
my_results.yellow_result

'00010'

In [425]:
print(my_results.myregexresult)

['SATIN' 'FLASK' 'TAUNT' 'FLANK' 'SALVO' 'FAITH' 'AMITY' 'CAULK' 'LOAMY'
 'COMMA' 'MAGIC' 'STALL' 'ALIGN' 'PLAZA' 'SLASH' 'CAPUT' 'ALOOF' 'MAXIM'
 'CLACK' 'TANGO' 'CHANT' 'CLANK' 'LOATH' 'FLASH' 'QUAIL' 'SCALP' 'SALON'
 'AWAIT' 'SLAIN' 'FLACK' 'NASTY' 'PASTA' 'PALSY' 'ALOFT' 'HASTY' 'FANCY'
 'CHAFF' 'TONGA' 'GAWKY' 'AXION' 'SASSY' 'AMONG' 'PANIC' 'LAUGH' 'MOCHA'
 'SMACK' 'GAMMA' 'NANNY' 'PLANT' 'ALPHA' 'SHACK' 'SWASH' 'GAMUT' 'SPANK'
 'GIANT' 'CLAIM' 'PIZZA' 'GAUNT' 'SALTY' 'ANVIL' 'KAPPA' 'GUAVA' 'PATSY'
 'NINJA' 'QUOTA' 'KOALA' 'ANNUL' 'MACHO' 'PASTY' 'SWATH' 'PSALM' 'SPAWN'
 'SAUNA' 'HAVOC' 'CHAMP' 'KNACK' 'WACKY' 'SCALY' 'VOILA' 'MAMMY' 'SCANT'
 'AWFUL' 'PATIO' 'AWASH' 'SHANK' 'MATCH' 'JAUNT' 'TAWNY' 'SNACK' 'CANON'
 'FAINT' 'VAULT' 'SALSA' 'WAGON' 'PANSY' 'JUNTA' 'STANK' 'SAPPY' 'CLASS'
 'AUNTY' 'STAIN' 'GAILY' 'FANNY' 'SOAPY' 'VAUNT' 'TACIT' 'SWAMP' 'TOAST'
 'GNASH' 'TAFFY' 'AGLOW' 'CHALK' 'MAFIA' 'ASCOT' 'ALLOW' 'PAINT' 'AMPLY'
 'COAST' 'SLANT' 'CACTI' 'SHAWL' 'CLANG' 'SHAFT' 'W

In [528]:
my_results.myregexresult

array(['SATIN', 'FLASK', 'TAUNT', 'FLANK', 'SALVO', 'FAITH', 'AMITY',
       'CAULK', 'LOAMY', 'COMMA', 'MAGIC', 'STALL', 'ALIGN', 'PLAZA',
       'SLASH', 'CAPUT', 'ALOOF', 'MAXIM', 'CLACK', 'TANGO', 'CHANT',
       'CLANK', 'LOATH', 'FLASH', 'QUAIL', 'SCALP', 'SALON', 'AWAIT',
       'SLAIN', 'FLACK', 'NASTY', 'PASTA', 'PALSY', 'ALOFT', 'HASTY',
       'FANCY', 'CHAFF', 'TONGA', 'GAWKY', 'AXION', 'SASSY', 'AMONG',
       'PANIC', 'LAUGH', 'MOCHA', 'SMACK', 'GAMMA', 'NANNY', 'PLANT',
       'ALPHA', 'SHACK', 'SWASH', 'GAMUT', 'SPANK', 'GIANT', 'CLAIM',
       'PIZZA', 'GAUNT', 'SALTY', 'ANVIL', 'KAPPA', 'GUAVA', 'PATSY',
       'NINJA', 'QUOTA', 'KOALA', 'ANNUL', 'MACHO', 'PASTY', 'SWATH',
       'PSALM', 'SPAWN', 'SAUNA', 'HAVOC', 'CHAMP', 'KNACK', 'WACKY',
       'SCALY', 'VOILA', 'MAMMY', 'SCANT', 'AWFUL', 'PATIO', 'AWASH',
       'SHANK', 'MATCH', 'JAUNT', 'TAWNY', 'SNACK', 'CANON', 'FAINT',
       'VAULT', 'SALSA', 'WAGON', 'PANSY', 'JUNTA', 'STANK', 'SAPPY',
       'CLASS', 'AUN

In [134]:
print(secret)

TSINE


In [529]:
print(my_results.mylength)

235


# This explores the np.vectorize function that the feedback script has been modified to support

In [241]:
np_vocab_reduction = np.vectorize(feedback)

In [530]:
my_secret = np_word_list[0]
print(my_secret)
timestart = time.time()
mylengtharray = np_vocab_reduction(np_word_list,secret = my_secret,printing=False)
timeend = time.time()
total_time = timeend - timestart
print(total_time,"seconds")

SATIN
8.509259462356567 seconds


In [196]:
print(total_time/60,"Minutes")

0.1462755759557088 Minutes


In [531]:
len(mylengtharray)

2315

In [532]:
mylengtharray[0:50]

array([  1, 524,  59,   5, 172,  53, 207,  36, 284,  78, 357,  26,  12,
         5, 459, 970,  76,  19, 208, 159, 237,   4, 169,  98, 103, 290,
        53,  21, 212, 238,  68,  16, 126, 129, 172, 218, 145,  11, 228,
         2,  84, 302, 116,  15,  13,  19, 146, 285,  24,  19])

In [245]:
np_word_list[mylengtharray == 1]

array(['SATIN', 'STERN', 'SALON', 'SLAIN', 'STRIP', 'STAIR', 'STEIN',
       'PATSY', 'STINT', 'BASIC', 'SCANT', 'PATIO', 'STONE', 'FAINT',
       'TRAIN', 'STANK', 'STAIN', 'SATYR', 'PAINT', 'SLANT', 'STAND',
       'WAIST', 'STING', 'STONY', 'UNTIE', 'BASIS', 'STUNT', 'BASIL',
       'BASIN', 'SAINT', 'STINK', 'STAID', 'UNTIL', 'TAINT', 'MASON',
       'RATIO', 'ARTSY', 'TITAN', 'SNAIL', 'ANTIC', 'CABIN'], dtype='<U5')

# This shows that the default function of feedback returns a simple length

In [252]:
feedback('GOYLE')

316

In [256]:
freq = defaultdict(dict)
    
alphabet = string.ascii_uppercase    
listabc=[c for c in alphabet]
npabc = np.array(listabc)
print("alphabet",alphabet)
print("listabc",listabc)
print("npabc",npabc)

alphabet ABCDEFGHIJKLMNOPQRSTUVWXYZ
listabc ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
npabc ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [262]:
npabc[0]

'A'

# alphabet frequency of vocabulary

In [449]:
# This function takes in a word list of vocabulary and returns a dataframe 
# with the frequency of each letter in the vocabulary by position
def alphabet_frequency(wordlist=np_word_list):
    global npabc
    global newdf
    newdf = pd.DataFrame(0, index=npabc, columns=range(5))

    for word in wordlist:
        for pos in range(len(word)):
            char = word[pos]
            newdf.at[char,pos] += 1
    return(newdf)

In [542]:
mydf = alphabet_frequency()
display(mydf)

,0,1,2,3,4
A,141,304,307,163,64
B,173,16,57,24,11
C,198,40,56,152,31
D,111,20,75,69,118
E,72,242,177,318,424
F,136,8,25,35,26
G,115,12,67,76,41
H,69,144,9,28,139
I,34,202,266,158,11
J,20,2,3,2,0


In [455]:
type(mydf)

pandas.core.frame.DataFrame

In [457]:
# example of runing the function of the output results from the feedback .myregexresult reduced vocabulary list
mydf = alphabet_frequency(my_results.myregexresult)
display(mydf)


,0,1,2,3,4
A,38,94,93,0,31
B,0,0,0,0,0
C,28,4,9,19,6
D,0,0,0,0,0
E,0,0,0,0,0
F,14,3,4,5,3
G,11,4,5,8,7
H,5,18,0,4,20
I,0,8,16,26,4
J,3,0,0,1,0


# Information Value iv function

In [483]:
# This function takes in a guess and evaluates the frequency value of each letter in the word 
# compared to the frequency of that letter in that position in the entire vocabulary, 
# returning the sum of the frequencies of the letters
# this function requires a guess and an input dataframe of the frequencies of letters in the vocabulary
def iv(guess,inputdf,printing=False):
    if printing == True: print(guess)
    if printing == True: print(inputdf)
    ivsum = 0
    for pos in range(len(guess)):
        char = guess[pos]
        val = inputdf.at[char,pos]
        if printing == True: print("char",char,"val",val)
        ivsum += val
    if printing == True: print("ivsumi",ivsum)
    return(ivsum)

In [513]:
# example of running the df against a guess on the whole vocabulary
iv("APPLE",mydf,printing=True)

APPLE
     0    1    2    3    4
A  141  304  307  163   64
B  173   16   57   24   11
C  198   40   56  152   31
D  111   20   75   69  118
E   72  242  177  318  424
F  136    8   25   35   26
G  115   12   67   76   41
H   69  144    9   28  139
I   34  202  266  158   11
J   20    2    3    2    0
K   20   10   12   55  113
L   88  201  112  162  156
M  107   38   61   68   42
N   37   87  139  182  130
O   41  279  244  132   58
P  142   61   58   50   56
Q   23    5    1    0    0
R  105  267  163  152  212
S  366   16   80  171   36
T  149   77  111  139  253
U   33  186  165   82    1
V   43   15   49   46    0
W   83   44   26   25   17
X    0   14   12    3    8
Y    6   23   29    3  364
Z    3    2   11   20    4
char A val 141
char P val 61
char P val 58
char L val 162
char E val 424
ivsumi 846


846

# vocab_value 

In [523]:
# this function takes in a list of vocabulary words and a dataframe of letter frequencies
# this function creates a myIV_value_array numoy array which is a value score for every word in the vocabulary list
# the function then reads the words in the vocabulary list and looks the character and position up in the frequency df
# these scores are added up and stopred in a numpy array as a value score for each word in the vocabulary
# 
# this function returns results as a named tuple with four numpy arrays
# "myIV_value_array","sorter","sorted_myIV_value_array","sorted_np_word_list"
#
# myIV_value_array: an unsorted numpy array of the sum of frequency values of the words
# 
# Sorter is a sorted list of indexes.  This is used to sort the numpy array in descending order highest value first
#
# sorted_myIV_value_array is a sorted array of the sum of frequency values of the words
#
# sorted_np_word_list is a sorted array of teh vocabulary words based on the sum of frequency values
#
# this is an example of how to run this function
# myresults = vocab_value(np_word_list,mydf)
# print(myresults.sorter)
# print(myresults.sorted_myIV_value_array)
# print(myresults.sorted_np_word_list)
#
#

def vocab_value(np_word_list,inputdf=mydf,printing=False):
    myIV_value_array = np.zeros(len(np_word_list))
    if printing == True: print("myIV_value_array",myIV_value_array)
    for i in range(len(np_word_list)):
        word = np_word_list[i]
        val = iv(word,inputdf,printing=False)
        myIV_value_array[i]=val
    if printing == True: print(myIV_value_array)
    if printing == True: print(myIV_value_array.max())
    sorter = myIV_value_array.argsort()[::-1]
    sorted_myIV_value_array = myIV_value_array[sorter]
    sorted_np_word_list = np_word_list[sorter]
#    return(sorted_myIV_value_array,sorted_np_word_list)

    myresults = namedtuple("myresults",["myIV_value_array","sorter","sorted_myIV_value_array","sorted_np_word_list"])
    results = myresults(myIV_value_array,sorter,sorted_myIV_value_array,sorted_np_word_list)
    return(results)


In [543]:
myresults = vocab_value(np_word_list,mydf)
print(myresults.sorter)
print(myresults.sorted_myIV_value_array)
print(myresults.sorted_np_word_list)

[2152 1661 2197 ... 1918 1403 1756]
[1437. 1411. 1409. ...  326.  318.  310.]
['SLATE' 'SAUCE' 'SLICE' ... 'ETHOS' 'INBOX' 'NYMPH']


In [545]:
myresults.sorted_myIV_value_array[0:50]

array([1437., 1411., 1409., 1403., 1398., 1393., 1392., 1382., 1381.,
       1378., 1371., 1366., 1360., 1352., 1351., 1344., 1342., 1342.,
       1339., 1336., 1336., 1335., 1332., 1330., 1329., 1326., 1326.,
       1325., 1324., 1323., 1322., 1320., 1318., 1315., 1315., 1313.,
       1312., 1311., 1310., 1310., 1310., 1309., 1309., 1308., 1299.,
       1299., 1299., 1296., 1293., 1293.])

In [546]:
myresults.sorted_np_word_list[0:50]

array(['SLATE', 'SAUCE', 'SLICE', 'SHALE', 'SAUTE', 'SHARE', 'SOOTY',
       'SHINE', 'SUITE', 'CRANE', 'SAINT', 'SOAPY', 'SHONE', 'SHIRE',
       'SAUCY', 'SLAVE', 'SENSE', 'CEASE', 'SANER', 'SNARE', 'STALE',
       'CRATE', 'SPREE', 'SHORE', 'SUAVE', 'SLIDE', 'STARE', 'SLIME',
       'SORRY', 'BRACE', 'SHINY', 'GOOEY', 'SEIZE', 'CRONE', 'SLEET',
       'STATE', 'BRINE', 'SCREE', 'SHADE', 'SPACE', 'SPARE', 'SLANT',
       'SHAME', 'SALLY', 'TRACE', 'SCALE', 'SPINE', 'SHAKE', 'TEASE',
       'STONE'], dtype='<U5')